## 実験の目的
- LLM(大規模言語モデル)が文脈に沿った回答を行えるようにすること
- 今回は「スプラトゥーン3」に関する情報を回答できることを目指します



## LangChainを利用
- 今回の実験ではライブラリはLangChainを利用します
- LangChainはLLM(大規模言語モデル)アプリケーションの開発のために便利な機能を提供してくれているツールです

## RAGの3つのステップ

1. **情報の準備 (データの読み込みとベクトル化)**
テキストデータを分割し、ベクトル空間にマッピングします。
2. **検索 (Retrieval)**
クエリに基づいて関連するデータをベクトル空間から検索します。
3. **生成 (Generation)**
検索したデータを文脈(context)として使用し、生成AIが回答を生成します。

## 1. 情報の準備 (データの読み込みとベクトル化)

### 生成AIに答えさせたい内容がまとまったデータを準備する
- 今回はWikiの「スプラトゥーン3」のページを利用します
- ページの取得は省略します

### テキストの読み込み

In [ ]:
from langchain_community.document_loaders import TextLoader

# テキストの読み込み
loader = TextLoader("output_wikipedia.txt") # 取得してきたファイルを指定

### DocumentLoader
- まずデータの読み込みに使えるツールがDocumentLoaderです
  - https://python.langchain.com/docs/integrations/document_loaders/
- Github, Slack, Notion, Excel, PDFなどさまざまな形式のファイルを読み込むための方法が提供されている
- 今回は単純なtxtファイルなのでTextLoaderを利用する

In [ ]:
# 実際に読み込まれたか確認
raw_docs = loader.load()
print("ドキュメントの数:", len(raw_docs))

In [ ]:
print(raw_docs)

### テキストの分割
- ドキュメントをある程度の長さでチャンクに分割する
  - チャンク：分割したテキストの1つ１つのこと
- ドキュメントを適切な大きさのチャンクに分割することで、LLMに入力するトークン数を削減したり、より正確な回答を得やすくなる場合がある
- 今回はCharacterTextSplitterを利用して先ほどの1つの文章を複数に分割していく

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_docs)
print("ドキュメントの数:", len(docs))

- 元々1個だったドキュメントが27個に分割された
- LangChainでは他にもソースコードをクラスや関数のようなまとまりでよしなに分割してくれる機能も提供されている

In [ ]:
# 0番目のドキュメント
print(docs[0])

In [ ]:
# 1番目のドキュメント
print(docs[1])

### テキストのベクトル化（ベクトルストアを作成する）
- ここではOpenAIのEmbedding APIを使いテキストをベクトル化します
- モデルはtext-embedding-3-smallを利用します

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Open AIのEmbedding APIをラップしたOpenAIEmbeddingsクラス
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### 実際にEmbedding modelを利用してテキストのベクトル化を試してみる

In [ ]:
query = "秩序の街とはどのような街でしょうか"
vector = embeddings.embed_query(query) # Open AI APIへリクエストが行われる
print("次元数: ", len(vector))
print(vector)

- 1536というベクトルの次元数は「text-embedding-3-small」というモデルの設計時に決まっている
- 次元が高いほど、多くの情報を保持できるが、計算コストも増えるのでバランスをとって1536次元になっているらしい

### ベクトルストアの作成
- 今回はローカルで気軽に利用できるベクトルストアとしてChromaを利用する
- ベクトルストアの作成は簡単にできるようになっており、ドキュメントをベクトルストアに保存する際に内部で勝手にベクトル化される
- ここでもベクトル化にはOpen AI APIへリクエストが行われる

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db", # ローカルの保存場所
)

# ベクトル化はVector storeのクラスにデータを保存する際に内部的に実行される
vector_store.add_documents(documents=docs, embeddings=embeddings)

In [ ]:
# vector storeに保存されたベクトルを確認してみる

# idの確認
doc1_id = vector_store.get(include=['embeddings', 'documents'])['ids'][0]
print(doc1_id)

In [ ]:
# ベクトルの値を確認
doc1_vector = vector_store.get(include=['embeddings', 'documents'])['embeddings'][0] # getでembeddingsはデフォルトで除外されるので、includeで指定する

print("次元数: ", len(doc1_vector))
print(doc1_vector)

In [ ]:
# 文章を確認
doc1 = vector_store.get(include=['embeddings', 'documents'])['documents'][0]
print(doc1)

## 2. 検索 (Retrieval)
### retrieverを作成
- ベクトルストアから関連するドキュメントを得るインターフェースを「retriever」と呼ぶ（検索をしてくれるもの）
- ベクトルストアのインスタンスからretrieverの作成はLangChain側で簡単にできるようになっている

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3}) # 最大3件の検索結果を返す

### 試しに質問に近い文章をベクトルストアから検索してみよう

In [ ]:
# 質問に近いドキュメントを検索
query = "スミナガシートに関して教えて"

context_docs = retriever.invoke(query)
print("検索した結果のドキュメント数: ", len(context_docs))

In [ ]:
context_docs[0]

## 3. 生成 (Generation)
### 検索結果を元にAIに回答を生成させる
- 検索結果をプロンプトに埋め込みLLMに質問して回答をもらう
- まず完成形を見せて、順番に解説します

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
)

output = chain.invoke("秩序の街とはどのような街でしょうか")
print(output)

### LCEL(LangChain Expression Language)について
- 一連の処理をLCELの記法で実装
- Runnableなクラスを「｜」で繋ぐことで新たなRunnableを作り、invokeしたときに内部のRunnableが順番にinvokeされる

```
chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
)
```

- LCELの記法に関して理解するため順番にinvokeしてみる

In [ ]:
prompt_value = prompt.invoke({"context": "テストの文脈です。明日の天気は晴れです。", "question": "明日の天気は何ですか？"})
print(prompt_value)

In [ ]:
ai_messsage = model.invoke(prompt_value)
print(ai_messsage)

In [ ]:
output_parser = StrOutputParser()
output = output_parser.invoke(ai_messsage)
print(output)